<a href="https://colab.research.google.com/github/arminsalmasi/technical-dashboard/blob/master/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install missing packages and import

In [14]:
# missing packages
import sys
import subprocess
import pkg_resources
required = {'yfinance','evidently','facets-overview'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    python = sys.executable
    subprocess.check_call([python,
                           '-m', 'pip', 'install',
                           *missing], 
                          stdout=subprocess.DEVNULL)

# imports
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import yfinance as yf
from datetime import date, datetime, timedelta, timezone
from statsmodels.tsa.seasonal import seasonal_decompose
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import copy
from IPython.display import clear_output

# third part packages should be enabled
from google.colab import output
output.enable_custom_widget_manager()
from ipywidgets import interact, interactive
from ipywidgets import widgets
from evidently.dashboard import Dashboard
from evidently.tabs import DataDriftTab

# initialize variables
* tickers: list of tickers (strings)

* start/end: begining/end of the history (datetime), 

* data collection interval: 
>1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo (default is '1d')

In [15]:
# initialize variables
ticks = ['msft']
names = ['Microsoft']
key = 'Close' 
startTime = '2020-01-01'
endTime = datetime.today()
interval = '1d'
startTimeSeas = '2010-01-01'
endTimeSeas = datetime.today() 
intervalSeas = '1mo'

#Downloads tickers data from yahoo finance

In [16]:
# daily data
dfDict={}
yf.pdr_override() 
for tick in ticks:
    dfDict[tick]=(pdr.get_data_yahoo(tick, 
                                     start = startTime, 
                                     end = endTime, 
                                     interval = interval))
    if len(dfDict[tick].index)==0:
      del dfDict[tick]
      c = ticks.count(tick)
      for i in range(c):
        ticks.remove(tick)

# monthly data for seasonality analysis and stock info
dfDictSeasonal = {}
dfDictSeasonalStockInfo = {}
yf.pdr_override() 
for tick in ticks:
    dfDictSeasonal[tick]=(pdr.get_data_yahoo(tick, 
                                  start = startTimeSeas, 
                                  end = endTimeSeas, 
                                  interval = intervalSeas))
    dfDictSeasonalStockInfo[tick] = yf.Ticker(tick)
    if len(dfDictSeasonal[tick].index)==0:
      del dfDictSeasonal[tick]
      c = ticks.count(tick)
      for i in range(c):
        ticks.remove(tick)
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Function definitions

In [18]:
def macd(df, key, macdPeriods=[12 , 9 , 26]):
  ''' returns macd of given key of the dataframe with designated periods
    input: 
    output: '''
  df['macd']  = EMA(df, 
                    key, 
                    macdPeriods[0]) - EMA(df,key,macdPeriods[1])
  df['macd_signal'] = EMA(df, 
                          'macd', 
                          macdPeriods[2])
  return df

def RSI(df, key, window=14, adjust=False):
  '''returns RSI of the given key in the dataframe      
    input: 
    output: '''
  delta = df[key].diff(1).dropna()
  loss = delta.copy()
  gains = delta.copy()
  gains[gains < 0] = 0
  loss[loss > 0] = 0
  gain_ewm = gains.ewm(com=window - 1, adjust=adjust).mean()
  loss_ewm = abs(loss.ewm(com=window - 1, adjust=adjust).mean())
  RS = gain_ewm / loss_ewm
  df['RSI'] = 100 - 100 / (1 + RS)
  return df

def EMA(data, key, period):
  ''' returns exponential moving average
    input: 
    output: '''
  return data[key].ewm(span=period, adjust=False).mean()

def SMA(data, key, period):
  '''returns simple moving average
    input: 
    output: '''
  return data[key].rolling(window=period).mean()   

def crossOvers(df, keyMA1, keyMA2, keyOut1, keyOut2):
    '''returns cross over/under points
    input: datafram, keyMA1, keyMA2 (keys of lines to detect cross overs )
    output: dataframe including keyOut1 and keyOut2 values (nan and 1)'''
    flag = -1
    for i,id in enumerate(df.index):
        if not(np.isnan(df[keyMA1][i])) and not(np.isnan(df[keyMA2][i])):
          if df[keyMA1][i] > df[keyMA2][i] :
            if flag != 1:
                df[keyOut1][i] = 1
                flag=1
          elif df[keyMA1][i] < df[keyMA2][i] :
            if flag !=0:
              df[keyOut2][i] = 1
              flag=0
    return df

def seasonalDecompose(df, key, period=12):
  '''returns seasonality based on the given period
    input: 
    output: '''
  analysis =df[key]
  x = np.arange(0, len(df.index.values))
  nbins = x.shape[0]//period
  decompose_result_mult = seasonal_decompose(analysis, 
                                             period=period)
  outdf = pd.DataFrame()
  outdf['Seasonal'] = decompose_result_mult.seasonal
  outdf['Trend'] = decompose_result_mult.trend
  outdf['Resid'] = decompose_result_mult.resid
  return outdf

def lastMASignal(df, key1, key2):
  '''returns the last signal of cross overs of key1 and key2
    input: 
    output: '''
  b,s='buy','sell'
  if df[key1][-1]>df[key2][-1]:
    print(f'{key1:>5}>{key2} {b :^10}')
    return True
  else:
    print(f'{key1:>5}<{key2} {s :^10}')
    return False

def lastSignal(df, key1, key2, tick, name):
  '''returns the last signal
      input: 
      output: boolean'''
  a = df[key1][df[key1]==1].index[-1]
  b = df[key2][df[key2]==1].index[-1]
  if a>b:
    print(f'{key1:<15}  last buy:  {a.date()}')
    return True
  else:
    print(f'{key2:<15} last sell: {b.date()}')
    return False

# [Moving average](https://www.investopedia.com/articles/active-trading/052014/how-use-moving-average-buy-stocks.asp)




In [19]:
# calcualte EMA and SMA lines and signals
MAPeriods = [(12,'EMA'), (22,'SMA'), (44,'SMA'), (260,'SMA')] 
for tick in ticks:
  for MAPeriod in MAPeriods:
    if MAPeriod[1] == 'SMA': 
      dfDict[tick][MAPeriod[1] + str(MAPeriod[0])] = SMA(dfDict[tick],
                                                             key, 
                                                             MAPeriod[0])
    if MAPeriod[1] == 'EMA': 
      dfDict[tick][MAPeriod[1] + str(MAPeriod[0])] = EMA(dfDict[tick], 
                                                             key, 
                                                             MAPeriod[0])
  # Find cross overs of first abd second periods
  keyMA1 = MAPeriods[0][1] + str(MAPeriods[0][0])
  keyMA2 = MAPeriods[1][1] + str(MAPeriods[1][0])
  newDict = {'MA_Buy_Signal'  : np.empty(len(dfDict[tick].index)),
             'MA_Sell_Signal' : np.empty(len(dfDict[tick].index))}
  newdf = pd.DataFrame(index=dfDict[tick].index, 
                       columns = newDict.keys())
  dfDict[tick] = pd.concat([dfDict[tick], newdf], axis = 1)
  dfDict[tick] = crossOvers(dfDict[tick], 
                            keyMA1, keyMA2,
                            'MA_Buy_Signal','MA_Sell_Signal')   

# [MACD](https://www.investopedia.com/terms/m/macd.asp)
* A 9-day EMA of the MACD called the "signal" on top of the MACD, 
trigers buy/sell. 

* buy when the MACD crosses above its line and sell/short if 
the MACD crosses below the signal. 

* Moving average convergence indicators can be 
interpreted in several ways, but the more common methods are crossovers, 
divergences, and rapid rises/falls.

In [20]:

macdPeriods = [12, 9, 26] #macd standard
for tick in ticks:
  dfDict[tick] = macd(dfDict[tick], 
                      'Close', 
                      macdPeriods)
  newDict = {'macd_Buy_Signal'  : np.empty(len(dfDict[tick].index)),
             'macd_Sell_Signal' : np.empty(len(dfDict[tick].index))}
  newdf = pd.DataFrame(index=dfDict[tick].index, 
                       columns = newDict.keys())
  dfDict[tick] = pd.concat([dfDict[tick], newdf], axis = 1)
  dfDict[tick] = crossOvers(dfDict[tick], 
                            'macd', 'macd_signal', 
                            'macd_Buy_Signal', 'macd_Sell_Signal')

# [RSI](https://www.investopedia.com/terms/r/rsi.asp)
* The standard number of periods used to calculate the initial RSI value is 14

* The RSI will rise with number and size of up days. 

* The RSI will fall as the number and size of down days increase.

* use RSI to predict the price behavior of a security.

* validate trends and trend reversals.

* A horizontal trendline between the levels of 30 and 70 when a **strong trend** is in place to better identify the overall trend and extremes.

* point to overbought and oversold securities.

* provide short-term buy and sell signals.

* Should be used with others to support trading strategies.

* RSI crosses 30(oversold- trading at a lower price than it should), a bullish sign 

* RSI crosses 70(overbought-priced above where it should be), a bearish sign.

* During an uptrend, the RSI tends to stay above 30 and should frequently hit 70.

* During a downtrend, it is rare to see the RSI exceed 70 andfrequently hits 30 or below.

* potential reversals. 
> 1.  if the RSI can’t reach 70 on a number of consecutive price swings during an uptrend but then drops below 30, the trend has weakened and could be reversing lower.
> 2.   If the downtrend is unable to reach 30 or below and then rallies above 70, that downtrend has weakened and could be reversing to the upside.
  
* The bullish divergence: when the RSI displays an oversold reading followed by a higher low, that appears with lower lows in the price chart. This may indicate rising bullish momentum, and a break above oversold territory could be used to trigger a new long position.

* The bearish divergence: when the RSI creates an overbought reading followed by a lower high that appears with higher highs on the price.

* A positive RSI reversal may take place (buy signal): 
> the RSI reaches a low that is lower than its previous low at the same time that the price reaches a low that is higher than its previous low price. 

* A negative RSI reversal (sell signal): 
> the RSI reaches a high that is higher that its previous high at the same time that the price reaches a lower high. This formation would be a bearish sign and a sell signal.

* True reversal signals are rare and can be difficult to separate from false alarms.

In [21]:
for tick in ticks:
  dfDict[tick] = RSI(dfDict[tick], key)

In [22]:
for tick,name in zip(ticks,names):
  flags =[]
  print(f'{name:<10} {tick:<10}')
  flags.append(lastSignal(dfDict[tick], 
                          'MA_Buy_Signal', 'MA_Sell_Signal', 
                          tick, name)) 
  flags.append(lastSignal(dfDict[tick], 
                          'macd_Buy_Signal', 'macd_Sell_Signal', 
                          tick, name)) 
  for i in range(len(MAPeriods)-1):
    flags.append(lastMASignal(dfDict[tick], 
                              MAPeriods[i][1]+str(MAPeriods[i][0]), 
                              MAPeriods[i+1][1]+str(MAPeriods[i+1][0])
                              )
    )
  print(f'RSI : {dfDict[tick].RSI[-1]}')
  #print(all(flags))

Microsoft  msft      
MA_Buy_Signal    last buy:  2023-01-20
macd_Sell_Signal last sell: 2023-01-13
EMA12>SMA22    buy    
SMA22>SMA44    buy    
SMA44<SMA260    sell   
RSI : 62.18978173049379
False


# Seasonality

In [23]:
seasonal = {}
for tick in ticks:
  seasonal[tick] = seasonalDecompose(dfDictSeasonal[tick], key)

# Plot EMA and SMA, buy sell signals, MACD, RSI, and Seasonality

In [28]:
for tick in ticks[:2]:
  min = seasonal[tick].Seasonal.min()
  max = seasonal[tick].Seasonal.max()
  #SMA-EMA
  fig1 = make_subplots(rows=2, cols=1, 
                       shared_xaxes = True, 
                       row_heights = [600,200])
  fig1.add_trace(go.Scatter(x = dfDict[tick].index,
                            y = dfDict[tick][key],
                            line = {"color":'rgb(153,153,153)'},
                            name = key),
                 row=1, col=1)
  for MAPeriod in MAPeriods:
    fig1.add_trace(go.Scatter(x = dfDict[tick].index,
                            y = dfDict[tick][MAPeriod[1]+str(MAPeriod[0])],
                            name = MAPeriod[1]+str(MAPeriod[0])),
                   row=1, col=1)
  fig1.add_trace(go.Scatter(mode = 'markers',
                            x = dfDict[tick][dfDict[tick].MA_Buy_Signal==1].index,
                            y = dfDict[tick][key][dfDict[tick].MA_Buy_Signal==1],
                            connectgaps = False,
                            marker_symbol = 5,
                            marker_size = 10,
                            marker_color = "green", 
                            name = 'EMA-SMA Buy'),
                 row=1, col=1)
  fig1.add_trace(go.Scatter(mode = 'markers',
                            x = dfDict[tick][dfDict[tick].MA_Sell_Signal==1].index,
                            y = dfDict[tick][key][dfDict[tick].MA_Sell_Signal==1],
                            connectgaps = False,
                            marker_symbol = 6, 
                            marker_size = 10,
                            marker_color = "red",
                            name = 'EMA-SMA Sell'),
                 row=1, col=1)
  # Volume
  fig1.add_trace(go.Bar(x = dfDict[tick].index,
                        y = dfDict[tick].Volume,name='Volume',
                        marker = {'color':'black'}),
                 row=2, col=1)
  fig1.update_layout(title=tick.upper(),
                     xaxis_title = 'DateTime',
                     yaxis_title = 'SMA-EMA',
                     width = 1400, height = 800,
                     font = dict(family = "Courier New, monospace",
                               size = 18,
                               color = "RebeccaPurple"
                               )
                     )
 
  #macd
  fig2 = make_subplots(rows = 1, cols = 1, 
                       shared_xaxes = False)
  fig2.add_trace(go.Scatter(x = dfDict[tick].index,
                            y = dfDict[tick].macd,
                            line = {"color":'rgb(153,153,153)'},
                            name = "macd line"),
                 row=1, col=1)

  fig2.add_trace(go.Scatter(x = dfDict[tick].index,
                            y = dfDict[tick].macd_signal,
                            line = {"color":'#FC1CBF'},
                            name= 'macd signal'),
                 row=1, col=1)
  fig2.add_trace(go.Scatter(mode='markers',
                          x = dfDict[tick].index[dfDict[tick].macd_Buy_Signal==1],
                          y = dfDict[tick].macd_signal[dfDict[tick].macd_Buy_Signal==1],
                          connectgaps = False,
                          marker_symbol = 5, 
                          marker_size = 10,
                          marker_color = "green", 
                          name = 'macd Buy'),
                 row=1, col=1)
  fig2.add_trace(go.Scatter(mode = 'markers',
                          x = dfDict[tick].index[dfDict[tick].macd_Sell_Signal==1],
                          y = dfDict[tick].macd_signal[dfDict[tick].macd_Sell_Signal==1],
                          connectgaps = False,
                          marker_symbol = 6, 
                          marker_size = 10,
                          marker_color = "red",
                          name = 'macd Sell'),
                 row=1, col=1)
  fig2.update_layout(title=tick.upper(),
                     xaxis_title = 'DateTime',
                     yaxis_title = 'MACD',
                     width = 1400 ,height = 300,
                     font=dict(family = "Courier New, monospace",
                               size = 18,
                               color = "RebeccaPurple"
                               )
                     )
  #RSI
  fig3 = make_subplots(rows=1, cols=1, 
                       shared_xaxes=False)
  fig3.add_trace(go.Scatter(x = dfDict[tick].index,
                            y = dfDict[tick].RSI, 
                            name = 'RSI'),
                 row=1,col=1)
  fig3.add_hline(y = 70)
  fig3.add_hline(y = 30)
  fig3.update_layout(title=tick.upper(),
                     xaxis_title = 'DateTime',
                     yaxis_title = 'RSI',
                     width = 1250, height = 300,
                     font = dict(family = "Courier New, monospace",
                               size = 18,
                               color = "RebeccaPurple"
                               )
                     )
  
  #seasonality
  fig4 = make_subplots(rows=1, cols=1, 
                       shared_xaxes=False)
  fig4.add_trace(go.Scatter(x = seasonal[tick].index,
                            y = seasonal[tick].Seasonal, 
                            name = 'Seasonal'),
                 row=1, col=1)
  y0 = np.zeros(len(seasonal[tick].Seasonal[seasonal[tick].Seasonal==min].index.date))
  fig4.add_trace(go.Scatter(mode='markers',
        x=seasonal[tick].Seasonal[seasonal[tick].Seasonal==min].index.date,
        y= y0 + min,
        marker_symbol = 5, 
        marker_size = 10,
        marker_color = "green",
        name = 'minimum, buy'),
        row=1, col=1)
  y0 = np.zeros(len(seasonal[tick].Seasonal[seasonal[tick].Seasonal==max].index.date))
  fig4.add_trace(go.Scatter(mode='markers',
        x=seasonal[tick].Seasonal[seasonal[tick].Seasonal==max].index.date,
        y= y0 + max,
        marker_symbol = 6, 
        marker_size = 10,
        marker_color = "red",
        name = 'maximum, sell'),
        row=1, col=1)
  fig4.update_layout(title=tick.upper(),
                     xaxis_title='DateTime',
                     yaxis_title='Seasonality',
                     width=1400,height=300,
                     font=dict(family="Courier New, monospace",
                               size=18,
                               color="RebeccaPurple")
                     )
  fig1.show()
  fig2.show()
  fig3.show()
  fig4.show()


In [25]:
for tick in ticks:
  fig5 = make_subplots(rows=3, cols=1, 
                       shared_xaxes = True,
                       row_heights = [200,200,200])
  fig5.add_trace(go.Scatter(mode = 'markers',
                            x=dfDictSeasonalStockInfo[tick].dividends.index,
                            y=dfDictSeasonalStockInfo[tick].dividends,
                            line={"color":'green'},
                            name='dividends'),
                 row=1, col=1)
  fig5.add_trace(go.Scatter(mode = 'markers',
                            x=dfDictSeasonalStockInfo[tick].splits.index,
                            y=dfDictSeasonalStockInfo[tick].splits,
                            line={"color":'blue'},
                            name='splits'),
                 row=2, col=1)
  fig5.add_trace(go.Scatter(mode = 'markers',
                            x=dfDictSeasonalStockInfo[tick].earnings_dates.index,
                            y=dfDictSeasonalStockInfo[tick].earnings_dates['EPS Estimate'],
                            line={"color":'rgb(153,153,153)'},
                            name='EPS Estimate'),
                 row=3, col=1)
  fig5.add_trace(go.Scatter(mode = 'markers',
                            x=dfDictSeasonalStockInfo[tick].earnings_dates.index,
                            y=dfDictSeasonalStockInfo[tick].earnings_dates['Reported EPS'],
                            line={"color":'red'},
                            name='Reported EPS'),
                 row=3, col=1)
  fig5.update_layout(title=tick.upper(),
                     xaxis_title='DateTime',
                     yaxis_title='Stock info',
                     width=1400,height=600,
                     font=dict(family="Courier New, monospace",
                               size=18,
                               color="RebeccaPurple")
                     )
  fig5.show()
  # # fast access to subset of stock info (opportunistic)
  for d in dfDictSeasonalStockInfo[tick].fast_info:
    print(d,dfDictSeasonalStockInfo[tick].fast_info[d])
  
  for d in dfDictSeasonalStockInfo[tick].news:
      print('\n', d['title'], '\n', d['link'])

currency USD
dayHigh 264.0899963378906
dayLow 260.6600036621094
exchange NMS
fiftyDayAverage 245.05459991455078
lastPrice 263.1000061035156
lastVolume 25760900
marketCap 1958463842271.75
open 261.5299987792969
previousClose 263.45
quoteType EQUITY
regularMarketPreviousClose 263.6199951171875
shares 7443800064
tenDayAverageVolume 34885250
threeMonthAverageVolume 30498538
timezone America/New_York
twoHundredDayAverage 253.79254959106444
yearChange -0.10813557253045551
yearHigh 315.95001220703125
yearLow 213.42999267578125

 ChatGPT Sparked an AI Craze. Investors Need a Long-Term Plan. 
 https://finance.yahoo.com/m/6f5e3b38-d26b-37fb-8d7e-0df89d28297f/chatgpt-sparked-an-ai-craze..html

 Microsoft and ChatGPT Won't Be Google Killers. Here's the 1 Company That Could Be, Though. 
 https://finance.yahoo.com/m/6b8dba41-ffff-3ec5-9aed-8aceace4bc0e/microsoft-and-chatgpt-won%27t.html

 ‘Code red’ at Google as Microsoft’s Bing is no longer the butt of the joke 
 https://finance.yahoo.com/news/code

# plot seasonality


In [26]:
# plot seasonality
for tick in ticks:
  min =seasonal[tick].Seasonal.min()
  max =seasonal[tick].Seasonal.max()
  fig = make_subplots(rows=5, cols=1, 
                      shared_xaxes=True,
                      row_heights = [500,150,150,150,150])
  fig.add_trace(go.Scatter(x = dfDictSeasonal[tick].index,
                           y = dfDictSeasonal[tick][key],
                           name = "Close $"),
                row=1, col=1)
  fig.add_trace(go.Scatter(mode = 'markers',
              x=seasonal[tick].Seasonal[seasonal[tick].Seasonal==min].index.date,
              y=dfDictSeasonal[tick][key][seasonal[tick].Seasonal==min],
              marker_symbol = 5, 
              marker_size = 10,
              marker_color = "green",
              name = 'minimum, buy'),
              row=1,col=1)
  fig.add_trace(go.Scatter(mode = 'markers',
              x=seasonal[tick].Seasonal[seasonal[tick].Seasonal==max].index.date,
              y=dfDictSeasonal[tick][key][seasonal[tick].Seasonal==max],
              marker_symbol = 6, 
              marker_size = 10,
              marker_color = "red",
              name = 'maximum, sell'),
              row=1, col=1)
  fig.add_trace(go.Scatter(x = seasonal[tick].index,
                           y = seasonal[tick].Trend,
                           name= 'Trend'),row=2,col=1)
  fig.add_trace(go.Scatter(x = seasonal[tick].index,
                          y = seasonal[tick].Seasonal, 
                          name = 'Seasonal'
                          ),
                row=4, col=1)
  fig.add_trace(go.Scatter(mode='markers',
                           x = seasonal[tick].index,
                           y = seasonal[tick].Resid, 
                           marker_symbol = 1,
                           name = 'Residues'),
                row=3,col=1)
  y0 = np.zeros(len(seasonal[tick].Seasonal[seasonal[tick].Seasonal==min].index.date))
  fig.add_trace(go.Scatter(
      mode = 'markers',
      x = seasonal[tick].Seasonal[seasonal[tick].Seasonal==min].index.date,
      y = y0 +min,
      marker_symbol = 5, 
      marker_size = 10,
      marker_color = "green",
      name = 'minimum, buy'),
      row=4,col=1)
  y0 = np.zeros(len(seasonal[tick].Seasonal[seasonal[tick].Seasonal==max].index.date))
  fig.add_trace(go.Scatter(
      mode = 'markers',
      x = seasonal[tick].Seasonal[seasonal[tick].Seasonal==max].index.date,
      y = y0 + max,
      marker_symbol = 6, 
      marker_size = 10,
      marker_color = "red",
      name = 'maximum, sell'),
      row=4, col=1)
  # Volume
  fig.add_trace(go.Bar(
      x=dfDictSeasonal[tick].index,y=dfDictSeasonal[tick].Volume,name='Volume',
      marker = {'color':'black'}),
      row=5, col=1)

  fig.update_layout(
      title=tick.upper(),
      xaxis_title='DateTime',
      yaxis_title='Seasonality',
      autosize=False,
      width=1400,
      height=1000,
      font=dict(
          family="Courier New, monospace",
          size=18,
          color="RebeccaPurple")
      )
  fig.show()     